In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import NHiTSModel
from darts.metrics import mse, rmse, r2_score, mae, smape
from darts.dataprocessing.transformers import Scaler
import torch
from pytorch_lightning.callbacks import Callback, EarlyStopping
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Configurações de visualização
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

In [2]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

df = pd.read_csv("../data/ground_station_clean.csv")

In [3]:
encoders = {
    "cyclic":{
        "past":["month","dayofyear","day","hour","minute"],
    },
    "transformer": Scaler()
}

In [4]:
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

# Criar diretório para salvar os resultados das iterações
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [6]:
def objective(trial):

    output_chunk_length = trial.suggest_int("output_chunk_length", 12, 72)
    input_chunk_length = trial.suggest_int("input_chunk_length", output_chunk_length + 1, output_chunk_length + 72)
    num_layers    = trial.suggest_int("num_layers", 1, 6)
    num_blocks    = trial.suggest_int("num_blocks", 1, 4)
    num_stacks    = trial.suggest_int("num_stacks", 1, 5)
    layer_widths    = trial.suggest_int("layer_widths", 32, 256)
    activation = trial.suggest_categorical("activation", ["ReLU", "GELU"])
    dropout = trial.suggest_float('dropout', 0.0, 0.3)
    batch_size = trial.suggest_int("batch_size", 16, 64)
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    forecast_horizon = trial.suggest_int("forecast_horizon", 1, 24)

    pruner = PatchedPruningCallback(trial, monitor="val_loss")
    early_stopper = EarlyStopping("val_loss", min_delta=0.001, patience=10, verbose=True)
    pl_trainer_kwargs = {
            "accelerator": "auto",
            "callbacks": [early_stopper, pruner],
    }
    n = int(len(df)*0.8)
    train_df_fold, val_df_fold = df.iloc[:n], df.iloc[n:]
    train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
    val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

    print(f"\nIniciando Trial {trial.number}")
    print(f"Hiperparâmetros: {trial.params}")
    print("\nTreinando o modelo...")
    print(f"Train set: {len(train_fold)} samples") # Usar len() para TimeSeries
    print(f"Validation set: {len(val_fold)} samples") # Usar len() para TimeSeries
    
    scaler = Scaler()
    scaler = scaler.fit(train_fold)
    train_scaled = scaler.transform(train_fold)
    val_scaled = scaler.transform(val_fold)

    # Definir work_dir e model_name para consistência
    # force_reset=True irá limpar work_dir/model_name antes de cada treinamento da iteração.
    _work_dir = "/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/" # Certifique-se que este caminho existe e é gravável
    _model_name = "model_optuna_temp" # Nome do modelo temporário para cada iteração

    # Criar o diretório _work_dir se não existir
    os.makedirs(_work_dir, exist_ok=True)

    model = NHiTSModel(
        model_name=_model_name,
        work_dir=_work_dir,
        input_chunk_length=input_chunk_length,
        output_chunk_length=output_chunk_length,
        n_epochs=30, # Considere tornar n_epochs um hiperparâmetro ou usar um valor maior com EarlyStopping
        batch_size=batch_size,
        num_layers=num_layers,
        num_blocks=num_blocks,
        num_stacks=num_stacks,
        layer_widths=layer_widths,
        activation=activation,
        dropout=dropout,
        add_encoders=encoders,
        pl_trainer_kwargs=pl_trainer_kwargs,
        loss_fn=torch.nn.HuberLoss(),
        lr_scheduler_cls=torch.optim.lr_scheduler.ReduceLROnPlateau,
        lr_scheduler_kwargs={"mode":"min", "factor":0.5, "patience":4, "min_lr":1e-6},
        save_checkpoints=True,    
        show_warnings=True,
        force_reset=True, # Importante para que cada iteração treine do zero com este model_name
        optimizer_kwargs={"lr": lr, "weight_decay": 1e-5},
    )
    model.fit(
        series=train_scaled,
        val_series=val_scaled,
        verbose=True # Pode ser False para menos output durante a otimização do Optuna
    )
    
    try:
        loaded_model = NHiTSModel.load_from_checkpoint(model_name=_model_name, work_dir=_work_dir, best=True)
        print(f"Modelo carregado do checkpoint para trial {trial.number}")
    except FileNotFoundError:
        print(f"Checkpoint não encontrado para {_model_name} em {_work_dir}. Usando o modelo treinado em memória.")
        loaded_model = model # Usar o modelo treinado se o carregamento falhar (fallback)


    forecasts = loaded_model.historical_forecasts(
        train_scaled,# Como no script original
        start=0.8, # Início da previsão no conjunto de treinamento
        forecast_horizon=forecast_horizon,
        stride=1, # Gera uma previsão em cada ponto de tempo possível
        retrain=False,
        verbose=True # Pode ser False
    ) 

    forecasts_t = scaler.inverse_transform(forecasts)
    s = scaler.inverse_transform(train_scaled).split_after(0.8)[1]
    metrics = {}
    print("Iniciando a verificação das séries temporais para constância...")
    try:
        for target in target_columns:
            metrics[target] = {
                'MSE': mse(s[target], forecasts_t[target]),
                'RMSE': rmse(s[target], forecasts_t[target]),
                'MAE': mae(s[target], forecasts_t[target]),
                'R2': r2_score(s[target], forecasts_t[target]),
                'SMAPE': smape(s[target], forecasts_t[target]),
            }
        metrics_df = pd.DataFrame(metrics).T
        print("\nMétricas de performance:")
        print(metrics_df)
    except Exception as e:
        print(e)

    overall_smape_val = smape(s, forecasts_t)
    print(f"O SMAPE desse fold foi {overall_smape_val}")

    trial_dict = {
        "trial_number": trial.number,
        "input_chunk_length": input_chunk_length,
        "output_chunk_length": output_chunk_length,
        "num_layers": num_layers,
        "num_blocks": num_blocks,
        "num_stacks": num_stacks,
        "layer_widths": layer_widths,
        "activation": activation,
        "dropout": dropout,
        "batch_size": batch_size,
        "lr": lr,
        "forecast_horizon": forecast_horizon,
        "overall_smape_val": overall_smape_val,
        "metrics": metrics_df.to_dict(),
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Resultados do trial {trial.number} salvos em {json_path}")
    return overall_smape_val if not np.isnan(overall_smape_val) else float("inf")



In [7]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [8]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=400, callbacks=[print_callback])

[I 2025-05-14 19:08:13,561] A new study created in memory with name: no-name-d060419f-dc28-409c-89e8-3bcf108a8139



Iniciando Trial 0
Hiperparâmetros: {'output_chunk_length': 25, 'input_chunk_length': 81, 'num_layers': 2, 'num_blocks': 4, 'num_stacks': 2, 'layer_widths': 69, 'activation': 'GELU', 'dropout': 0.05162129052063117, 'batch_size': 50, 'lr': 0.0036924358748126706, 'forecast_horizon': 18}

Treinando o modelo...
Train set: 6647 samples
Validation set: 1662 samples


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | HuberLoss        | 0      | train
1 | train_criterion | HuberLoss        | 0      | train
2 | val_criterion   | HuberLoss        | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 1.0 M  | train
-------------------------------------------------------------
928 K     Trainable params
102 K     Non-trainable params
1.0 M     Total params
4.121     Total estimated model params size (MB)
90        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


...

Modelo carregado do checkpoint para trial 399


/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

[I 2025-05-14 21:52:28,079] Trial 399 finished with value: 47.56610847796094 and parameters: {'output_chunk_length': 12, 'input_chunk_length': 14, 'num_layers': 1, 'num_blocks': 3, 'num_stacks': 2, 'layer_widths': 120, 'activation': 'GELU', 'dropout': 0.22505910855817207, 'batch_size': 20, 'lr': 0.0008050833208917502, 'forecast_horizon': 1}. Best is trial 343 with value: 46.745670426740844.


Iniciando a verificação das séries temporais para constância...

Métricas de performance:
                                    MSE        RMSE        MAE          R2  \
Temperature                    0.899424    0.948380   0.709142    0.735801   
Precipitation_accumulated    103.097328   10.153685   7.964948 -107.621425   
Humidity                       6.979087    2.641796   2.098308    0.892677   
Wind_Speed_kmh                 0.107826    0.328369   0.142446    0.683151   
Soil_Moisture              37830.469593  194.500565  96.118234    0.948876   
Soil_Temperature               0.832774    0.912564   0.685041    0.851614   
Wind_Dir_Sin                   0.055063    0.234655   0.119298    0.657184   
Wind_Dir_Cos                   0.056352    0.237385   0.133139    0.803717   

                                SMAPE  
Temperature                 13.169311  
Precipitation_accumulated    1.755275  
Humidity                     2.517309  
Wind_Speed_kmh             158.188533  
Soil_Mo

In [9]:
print(f"Melhor Valor (Mínimo): {study.best_value}")
print(f"Melhores Parâmetros: {study.best_params}")

Melhor Valor (Mínimo): 46.745670426740844
Melhores Parâmetros: {'output_chunk_length': 14, 'input_chunk_length': 17, 'num_layers': 1, 'num_blocks': 3, 'num_stacks': 2, 'layer_widths': 173, 'activation': 'GELU', 'dropout': 0.25275218141405936, 'batch_size': 26, 'lr': 0.0006811809549645802, 'forecast_horizon': 1}


In [10]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Melhores resultados salvos em {json_path}")

Melhores resultados salvos em optuna_iteration_metrics/best_trial.json
